# Summary 

## Lecture 05 - Logistic Regression Classification Lab

```
by Seokkyu Kong
Date: 2016-05-05
```

참고 자료: 1 [모두를 위한 머신러닝/딥러닝 강의 - 홍콩과기대 김성훈교수님](http://hunkim.github.io/ml/)

참고 자료: 2, ML lab 05 - logistic regression classification을 TensorFlow에서 구현하기  
https://www.youtube.com/watch?v=t7Y9luCNzzE&feature=youtu.be

기존 랩에서는 train.txt 테스트 데이터를 사용한다. 여기서는 코세라의 연습문제 2번 자료를 이용해서 동일한 결과를 얻는지 비교해 본다.

따라서 기존 예제와 달리 아래 내용을 변경해본다.

- training dataset 은 mxn 으로 읽어서 처리한다. 
- bias term을 포함시킨다.
- 정규화를 적용한다. featureNormalize()는 코세라 연습문제에서 빌어온다.
- 새로운 test dataset을 예측하기 위해 placeholder를 사용한다.


In [1]:
%pylab inline

import numpy as np
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


In [2]:
def featureNormalize(X):
    """
    피쳐 정규화 함수: 각각의 피쳐는 단위가 다르기 때문에 평균이 0이고 표준편차가 1인
    정규화 값으로 변환된다.
    """
    X_norm = X
    n = np.size(X, 1)
    mu = np.zeros((1, n))
    sigma = np.zeros((1, n))
    
    # 0은 column 단위 연산, 1은 row 단위 연산 적용
    m = np.size(X, 0)
    mu = mean(X_norm, 0)
    sigma = std(X_norm, 0)
    
    X_norm = (X_norm - mu) / sigma
    
    return X_norm, mu, sigma




In [4]:
## Part 1: 데이터 로드
# -------------------------
xy = np.loadtxt('ex2data1.txt', delimiter=',')
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

m = x_data.shape[0]

# XXX: tensorflow는 x 입력값을 normalize 하는게 중요하다.
# XXX: 이것때문에 하루 날렸다. -_-;
x_data, mu, sigma = featureNormalize(x_data)


# bias 1 column 벡터 추가
x_data = np.column_stack((np.ones(m), x_data)) # x_data: m x n
y_data = y_data.reshape((m, 1)) # y_data: m x 1

n = x_data.shape[1]

#print x_data[0:5, :]
#print y_data[0:5, :]
#print m, n


## Part 2: 변수, 가설함수, cost function 및 gradient descent 정의
# ----------------------------------------------------------------
# X, Y 입력 변수 설정
X = tf.placeholder(tf.float32, [None, n])
Y = tf.placeholder(tf.float32, [None, 1])

# 가중치 변수 설정
W = tf.Variable(tf.random_uniform([n, 1], -1, 1)) # W: n x 1

# 가설 함수 구성
h = tf.matmul(X, W)
#hypothesis = tf.div(1., 1. + tf.exp(-h)) # OK, XXX: float32 계산으로 맞추기 위해서 1. 을 사용한다.
hypothesis = 1. / (1. + tf.exp(-h))


# cost function 구성
# XXX: +- 부호 구분을 잘해야 한다. 이것때문에 하루 종일 헤맸다. -_-;
# XXX: lab에서는 - 부호를 맨 앞으로 빼고 안의 수식은 +로 대체했다.
cost = tf.reduce_mean(-Y * tf.log(hypothesis) - (1 - Y) * tf.log(1 - hypothesis))


# gradient descent 정의
a = tf.Variable(0.1) # learning rate
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)


# 변수 초기화 및 그래프 시작
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)


# debug
#print "X: ", tf.Print(X, [X])
#print "Y: ", tf.Print(Y, [Y])
#print "W: ", tf.Print(W, [W])
#print "hypothesis: ", tf.Print(hypothesis, [hypothesis])
#print "cost: ", tf.Print(cost, [cost])
#print

#W = tf.Variable(np.zeros([n, 1]))
#W = tf.Variable([[-25.1613187],    [0.20623159],   [0.20147149]])
#print sess.run(hypothesis, feed_dict={X: [[0, 0, 0]]})
#print sess.run(train, feed_dict={X: [[0, 0, 0]], Y: [[0]]})
#print sess.run(cost, feed_dict={X: [[0, 0, 0]], Y: [[0]]})
#print sess.run(train, feed_dict={X: [[0, 0, 0]], Y: [[0]]})


## Part 3: training dataset
# ------------------------------
for step in xrange(6001):
    sess.run(train, feed_dict = {X: x_data, Y: y_data})
    if step % 1000 == 0:
        print step, sess.run(cost, feed_dict = {X: x_data, Y: y_data}), sess.run(W)
    

## Part 4: training set에 대한 모델의 정확도를 계산한다.
# -----------------------------------------------------
# exam 1의 점수 45점과 exam 2의 점수 85점을 받은 학생에 대해서 예측한다.
# XXX: 입력 자료는 항상 normalize 해야 한다.
newdata = np.array([45., 85.])
newdata = (newdata - mu) / sigma # normalize
newdata = np.append(1, newdata) # bias term 추가
newdata = newdata.reshape(1, 3) # 1x3 dimension
prob = sess.run(hypothesis, feed_dict = {X: newdata}) # predict
print('For a student with scores 45 and 85, we predict an admission probability of %f' % prob)


# XXX: 계산된 p 값은 차원이 100x1이다. y_data: 100x1
p = sess.run(hypothesis, feed_dict = {X: x_data}) >= 0.5 # p: 100 x 1
print('Train Accuracy: %f' % (np.mean(double(p == y_data) * 100)))



0 1.1096 [[ 0.14191668]
 [-0.71575236]
 [-0.53954476]]
1000 0.210739 [[ 1.26506793]
 [ 3.03555417]
 [ 2.80028582]]
2000 0.204949 [[ 1.50311053]
 [ 3.53548336]
 [ 3.28282404]]
3000 0.203873 [[ 1.60632622]
 [ 3.7540462 ]
 [ 3.49415898]]
4000 0.203605 [[ 1.65790582]
 [ 3.86370397]
 [ 3.60023761]]
5000 0.20353 [[ 1.68518412]
 [ 3.92181396]
 [ 3.65646052]]
6000 0.203507 [[ 1.70000648]
 [ 3.95342374]
 [ 3.68704629]]
For a student with scores 45 and 85, we predict an admission probability of 0.773648
Train Accuracy: 89.000000


## 결론

코세라 연습문제의 정확도가 89% 나왔다. 동일한 결론을 얻었다.

In [69]:
#xy = np.loadtxt('lab05-train.txt', unpack=True, dtype='float32')
#xy = np.loadtxt('ex2data1.txt', unpack=True, dtype='float32', delimiter=',')
#x_data = xy[0:-1]
#y_data = xy[-1]

#n = x_data.shape[1]

#x_data = np.hstack((np.ones(n), x_data))

xy = np.loadtxt('ex2data1.txt', delimiter=',')
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

x_data, mu, sigma = featureNormalize(x_data)

x_data = np.column_stack((np.ones(x_data.shape[0]), x_data))
x_data = x_data.T

print "x_data: ", x_data.shape

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([1, len(x_data)], -1.0, 1.0))

h = tf.matmul(W, X)
hypothesis = tf.div(1., 1. + tf.exp(-h))
cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))

a = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

# debug
#print "X: ", tf.Print(X, [X])
#print "Y: ", tf.Print(Y, [Y])
#print "W: ", tf.Print(W, [W])
#print "h: ", tf.Print(h, [h])
#print "hypothesis: ", tf.Print(hypothesis, [hypothesis])
#print "cost: ", tf.Print(cost, [cost])
#print

for step in xrange(6001):
    sess.run(train, feed_dict = {X: x_data, Y: y_data})
    if step % 1000 == 0:
        print step, sess.run(cost, feed_dict={X: x_data, Y:y_data}), sess.run(W)

# training set에 대한 모델의 정확도를 계산한다.
p = sess.run(hypothesis, feed_dict = {X: x_data, Y: y_data}) >= 0.5

#y_data2 = y_data.flatten().astype(int)
print('Train Accuracy: %f' % (np.mean(double(p == y_data) * 100)))


x_data:  (3, 100)
0 0.892219 [[-0.49288213 -0.43764073  0.10852837]]
1000 0.210631 [[ 1.26809251  3.04188609  2.80638504]]
2000 0.204933 [[ 1.50428498  3.53796244  3.28522062]]
3000 0.203869 [[ 1.60688984  3.75524306  3.49531674]]
4000 0.203604 [[ 1.65819836  3.86432648  3.60083985]]
5000 0.203529 [[ 1.6853416   3.92214966  3.65678549]]
6000 0.203507 [[ 1.70009303  3.95360827  3.68722463]]
Train Accuracy: 89.000000
